# DMRG 

In this notebook we go through using TN4QA's DMRG solver.

In [15]:
from tn4qa.dmrg import DMRG
from tn4qa.utils import ham_dict_from_scf

import pyscf
import numpy as np

## Fermionic Hamiltonian

We can run DMRG directly on a second quantised Hamiltonian.

In [13]:
# Define H2 molecule
mol = pyscf.gto.Mole()
mol.atom = "H 0 0 0; H 0 0 1.0"
mol.basis = "sto-3g"
mol.spin = 0
mol.build()

# RHF calculation
mf = pyscf.scf.RHF(mol)
mf.kernel()

fci_energy = pyscf.fci.FCI(mf).run()
print(fci_energy.e_tot)

converged SCF energy = -1.06610864931794


np.float64(-1.0661086493179366)

In [14]:
import logging
from typing import Tuple

logger = logging.getLogger(__name__)


def _one_body_integrals(self) -> np.ndarray:
    """Get the one electron integrals."""
    logger.debug("Calculating one body integrals.")
    c_matrix_active = self.scf_method.mo_coeff
    logger.debug(f"{c_matrix_active.shape=}")
    logger.debug(f"{c_matrix_active[0].shape=}")

    logger.debug(f"{self.scf_method.get_hcore().shape=}")

    # Embedding procedure creates two different hcores
    # Using different v_eff
    if self.scf_method.get_hcore().ndim == 2:
        # Driver has not been used.
        hcore = [self.scf_method.get_hcore()] * 2
    elif self.scf_method.get_hcore().ndim == 3:
        # Driver has been used.
        hcore = self.scf_method.get_hcore()

    # one body terms
    if not self._restricted:
        logger.info("Calculating unrestricted one body intergrals.")
        one_body_integrals_alpha = c_matrix_active[0].T @ hcore[0] @ c_matrix_active[0]
        one_body_integrals_beta = c_matrix_active[1].T @ hcore[1] @ c_matrix_active[1]

        one_body_integrals = np.array(
            [one_body_integrals_alpha, one_body_integrals_beta]
        )

    else:
        logger.info("Calculating restricted one body integrals.")
        # We double these up so that we have the same number as
        # the unrestricted case.
        one_body_integrals = np.array(
            [c_matrix_active.T @ self.scf_method.get_hcore() @ c_matrix_active] * 2
        )
    logger.debug("One body integrals found.")
    logger.debug(f"{one_body_integrals.shape}")

    return one_body_integrals


def _two_body_integrals(self) -> np.ndarray:
    """Get the two electron integrals."""
    logger.debug("Calculating two body integrals.")
    c_matrix_active = self.scf_method.mo_coeff

    if not self._restricted:
        n_orbs_alpha = c_matrix_active[0].shape[1]
        n_orbs_beta = c_matrix_active[1].shape[1]

        # Could make this more flexible later.
        if n_orbs_alpha != n_orbs_beta:
            raise ValueError(
                "Must localize the same number of alpha and beta orbitals."
            )

        c_alpha = c_matrix_active[0]
        c_beta = c_matrix_active[1]

        # Pyscf is in chemist notation
        # later we transpose to physicist notation for openfermion
        spin_options = {
            "aaaa": (c_alpha, c_alpha, c_alpha, c_alpha),
            "bbbb": (c_beta, c_beta, c_beta, c_beta),
            "aabb": (c_alpha, c_alpha, c_beta, c_beta),
            "bbaa": (c_beta, c_beta, c_alpha, c_alpha),
        }

        two_body_integrals = []
        for spin in spin_options:
            two_body_compressed = pyscf.ao2mo.kernel(self.scf_method.mol, spin_options[spin])
            eri = pyscf.ao2mo.restore(1, two_body_compressed, n_orbs_alpha)
            two_body_integrals.append(np.asarray(eri.transpose(0, 2, 3, 1), order="C"))

    else:
        n_orbs = c_matrix_active.shape[1]

        two_body_compressed = pyscf.ao2mo.kernel(self.scf_method.mol, c_matrix_active)

        # get electron repulsion integrals
        eri = pyscf.ao2mo.restore(1, two_body_compressed, n_orbs)  # no permutation symmetry

        # Copy this 4 times so that we have the same number as
        # the unrestricted case
        # Openfermion uses physicist notation whereas pyscf uses chemists
        two_body_integrals = [np.asarray(eri.transpose(0, 2, 3, 1), order="C")] * 4

    two_body_integrals = np.array(two_body_integrals)

    logger.debug("Two body integrals found.")
    logger.debug(f"{two_body_integrals.shape}")
    return two_body_integrals


def _spinorb_from_spatial(
    one_body_integrals: np.ndarray, two_body_integrals: np.ndarray
) -> Tuple[np.ndarray, np.ndarray]:
    """Convert spatial integrals to spin-orbital integrals.

    Args:
        one_body_integrals (np.ndarray): One-electron integrals in physicist notation.
        two_body_integrals (np.ndarray): Two-electron integrals in physicist notation.

    Returns:
        one_body_coefficients (np.ndarray): One-electron coefficients in spinorb form.
        two_body_coefficients (np.ndarray): Two-electron coefficients in spinorb form.

    """
    logger.debug("Converting to spin-orbital coefficients.")
    n_qubits = one_body_integrals[0].shape[0] + one_body_integrals[1].shape[0]

    # Initialize Hamiltonian coefficients.
    one_body_coefficients = np.zeros((n_qubits, n_qubits))
    two_body_coefficients = np.zeros((n_qubits, n_qubits, n_qubits, n_qubits))

    # Loop through integrals.
    for p in range(n_qubits // 2):
        for q in range(n_qubits // 2):
            # Populate 1-body coefficients. Require p and q have same spin.
            one_body_coefficients[2 * p, 2 * q] = one_body_integrals[0, p, q]
            one_body_coefficients[2 * p + 1, 2 * q + 1] = one_body_integrals[1, p, q]

            # Continue looping to prepare 2-body coefficients.
            # Assumes 2e ints are ordered as aaaa,bbbb,aabb,bbaa.
            for r in range(n_qubits // 2):
                for s in range(n_qubits // 2):
                    # Same spin
                    two_body_coefficients[2 * p, 2 * q, 2 * r, 2 * s] = (
                        two_body_integrals[0, p, q, r, s]
                    )
                    two_body_coefficients[
                        2 * p + 1, 2 * q + 1, 2 * r + 1, 2 * s + 1
                    ] = two_body_integrals[1, p, q, r, s]

                    # Mixed spin in physicist
                    two_body_coefficients[2 * p, 2 * q + 1, 2 * r + 1, 2 * s] = (
                        two_body_integrals[2, p, q, r, s]
                    )
                    two_body_coefficients[2 * p + 1, 2 * q, 2 * r, 2 * s + 1] = (
                        two_body_integrals[3, p, q, r, s]
                    )

    # Truncate.
    one_body_coefficients[np.absolute(one_body_coefficients) < 1e-8] = 0.0
    two_body_coefficients[np.absolute(two_body_coefficients) < 1e-8] = 0.0

    return one_body_coefficients, two_body_coefficients


class MockBuilder:
    def __init__(self, scf_method):
        self.scf_method = scf_method
        self._restricted = True


mb = MockBuilder(mf)

ones = _one_body_integrals(mb)
twos = _two_body_integrals(mb)
ones, twos = _spinorb_from_spatial(ones, twos)
qubit_labels = {i for i in range(ones.shape[0])}
nuclear_energy = mf.energy_nuc()

In [15]:
####
# Next we run a fermionic DMRG calculation
####

# Expected FCI energy is about -1.13
for _ in range(1):
    dmrg = DMRG(
        hamiltonian=(ones, twos),
        max_mps_bond=32,
        method="two-site",
        hamiltonian_type="fermionic",
    )
    dmrg.run(maxiter=10)
    print(dmrg.energy + mf.energy_nuc())

-1.1011503302326182


In [4]:
ham_dict = ham_dict_from_scf(mf)
print(ham_dict)

[[ 0.031+0.j -0.   +0.j -0.   +0.j  0.   -0.j]
 [-0.   -0.j  0.   -0.j  0.   +0.j  0.   -0.j]
 [-0.   -0.j  0.   -0.j  0.   +0.j -0.   -0.j]
 [ 0.   +0.j  0.   +0.j -0.   +0.j  0.969+0.j]]
(0.031+0j) 3 3
(-0.173-0j) 3 12
(-0.173+0j) 12 3
(0.969+0j) 12 12


In [17]:
dmrg = DMRG(ham_dict, 8)
dmrg.run(10)
print(dmrg.energy)

-1.101150330232619
